<a href="https://colab.research.google.com/github/Aniebiet1/Explicit-Content-Detector/blob/main/Text_Explicit_detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Datset-link:https://www.kaggle.com/datasets/thedevastator/hate-speech-and-offensive-language-detection


# importing libraries

In [46]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

In [47]:
import re
import nltk
nltk.download('stopwords')
from nltk.util import pr
stemmer = nltk.SnowballStemmer("english")
from nltk.corpus import stopwords
import string
stopword = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Data Preprocessing

In [48]:
df = pd.read_csv("/content/Explicit_trainst.csv")

In [49]:
print(df.head())

   count  hate_speech_count  offensive_language_count  neither_count  class  \
0      3                  0                         0              3      2   
1      3                  0                         3              0      1   
2      3                  0                         3              0      1   
3      3                  0                         2              1      1   
4      6                  0                         6              0      1   

                                               tweet  
0  !!! RT @mayasolovely: As a woman you shouldn't...  
1  !!!!! RT @mleew17: boy dats cold...tyga dwn ba...  
2  !!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...  
3  !!!!!!!!! RT @C_G_Anderson: @viva_based she lo...  
4  !!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...  


In [50]:
df.describe()

,count,hate_speech_count,offensive_language_count,neither_count,class
count,24783.000000,24783.000000,24783.000000,24783.000000,24783.000000
mean,3.243473,0.280515,2.413711,0.549247,1.110277
std,0.883060,0.631851,1.399459,1.113299,0.462089
min,3.000000,0.000000,0.000000,0.000000,0.000000
25%,3.000000,0.000000,2.000000,0.000000,1.000000
50%,3.000000,0.000000,3.000000,0.000000,1.000000
75%,3.000000,0.000000,3.000000,0.000000,1.000000
max,9.000000,7.000000,9.000000,9.000000,2.000000


In [51]:
df["label"] = df["class"].map({0:"Hate Speech detected", 1: "Offensive Langueage detected", 2: "No Hate speech or Offensive language detected"})
df.head()

,count,hate_speech_count,offensive_language_count,neither_count,class,tweet,label
0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...,No Hate speech or Offensive language detected
1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,Offensive Langueage detected
2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,Offensive Langueage detected
3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,Offensive Langueage detected
4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,Offensive Langueage detected


In [52]:
df.isnull().any()

,0
count,False
hate_speech_count,False
offensive_language_count,False
neither_count,False
class,False
tweet,False
label,False


In [53]:
df = df[["tweet", "label"]]
df.head()

,tweet,label
0,!!! RT @mayasolovely: As a woman you shouldn't...,No Hate speech or Offensive language detected
1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,Offensive Langueage detected
2,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,Offensive Langueage detected
3,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,Offensive Langueage detected
4,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,Offensive Langueage detected


In [54]:
import re

def clean(text):
  text = str(text).lower()
  text = re.sub('\[.*?\]', '', text )
  text = re.sub('https?://\S+|www\.\S+', '', text)
  text = re.sub('<.*?>+', '', text)
  text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
  text = re.sub('\n', '', text)
  text = re.sub('\w*\d\w*', '', text)
  text = [word for word in text.split(' ') if word not in stopword]
  text = " ".join(text)
  return text

df["tweet"] = df["tweet"].apply(clean)
print(df.head())



                                               tweet  \
0   rt mayasolovely woman shouldnt complain clean...   
1   rt  boy dats coldtyga dwn bad cuffin dat hoe ...   
2   rt urkindofbrand dawg rt  ever fuck bitch sta...   
3           rt cganderson vivabased look like tranny   
4   rt shenikaroberts shit hear might true might ...   

                                           label  
0  No Hate speech or Offensive language detected  
1                   Offensive Langueage detected  
2                   Offensive Langueage detected  
3                   Offensive Langueage detected  
4                   Offensive Langueage detected  


# Spliting dataset in to train and test set
## Training the model

In [55]:
x = np.array(df["tweet"])
y = np.array(df["label"])

cv = CountVectorizer()
x = cv.fit_transform(x)
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.2, random_state = 60)
clf = DecisionTreeClassifier()
clf.fit(x_train, y_train)


DecisionTreeClassifier()

In [56]:
test_data = "I will kill you"
df = cv.transform([test_data]).toarray()
print(clf.predict(df))

['Hate Speech detected']


In [57]:
test_data = "I like u"
df = cv.transform([test_data]).toarray()
print(clf.predict(df))

['Offensive Langueage detected']


In [58]:
test_data = "You are a piece of shit"
df = cv.transform([test_data]).toarray()
print(clf.predict(df))

['Offensive Langueage detected']


In [59]:
test_data = "You are bad I dont like u"
df = cv.transform([test_data]).toarray()
print(clf.predict(df))

['Offensive Langueage detected']
